In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
from keras.preprocessing import image
pd.options.display.max_columns = 50

# Retinal Disease Classification: Multiclass Classification
This notebook seeks to classify diseased eyes through use of a computer vision model in TensorFlow.

We'll start off by loading our data into a pandas dataframe and adding the paths to the image files. Since it's already divided into train, validate, and test directories we'll start with making a dataframe for each.

In [ ]:
train_df = pd.read_csv('/kaggle/input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv')
val_df = pd.read_csv('/kaggle/input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv')
test_df = pd.read_csv('/kaggle/input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv')

In [ ]:
train_df['img_path'] = [f'/kaggle/input/retinal-disease-classification/Training_Set/Training_Set/Training/{id}.png' for id in train_df['ID']]
val_df['img_path'] = [f'/kaggle/input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/{id}.png' for id in val_df['ID']]
test_df['img_path'] = [f'/kaggle/input/retinal-disease-classification/Test_Set/Test_Set/Test/{id}.png' for id in test_df['ID']]

# Exploration and Feature Engineering

In [ ]:
train_df = train_df.drop(labels=['ID'],axis=1)

In [ ]:
columns = list(train_df.columns)
columns.remove('img_path')
d_total = 0
for col in columns:
    print(col)
    print(train_df[col].value_counts())
    print('----------------')
    if col != 'Disease_Risk':
        d_total += train_df[col].sum()
        
print(d_total)


## Some notes from this:
* We can see that ODPM and HR have no positive examples so we will drop the columns.
* Since we're trying to predict specific diseases, we'll drop Disease_Risk to help with imbalancing.
* A lot of the columns have very few examples, so we will have to do some oversampling and use class weights.
* Some images are representative of more than one disease, since d_total is greater than the total number of images. We'll need to do multilabel classification.

In [ ]:
train_df = train_df.drop(labels=['Disease_Risk','ODPM', 'HR'], axis=1)
val_df = val_df.drop(labels=['ID', 'Disease_Risk', 'ODPM', 'HR'], axis=1)
test_df = test_df.drop(labels=['ID', 'Disease_Risk', 'ODPM', 'HR'], axis=1)

DR has our highest count for diseases, so we'll use that number as our target for rebalancing the dataframe with oversampling.

In [ ]:
train_df.columns

In [ ]:
train_df[train_df['PTCR']==1]

In [ ]:
def weight_calc(col):
    total = len(train_df)
    weight = (1 / train_df[col].sum()) *  total / 2
    return weight

In [ ]:
Y_train = list(train_df.drop(['img_path'], axis=1).columns)
Y_val = list(val_df.drop(['img_path'], axis=1).columns)
Y_test = list(test_df.drop(['img_path'], axis=1).columns)
unq_disease = len(Y_train)

In [ ]:
# Was going to implement class weights for this solution, but TF currently does not support using class weights in its metrics.
# class_weights = {}
# for i in range(0, unq_disease):
#     class_weights[f'{i}'] = weight_calc(Y_train[i])

We'll display some images just to get an idea of what kind of data we're working with.

In [ ]:
plt.subplots(3, 4, figsize=(240, 160))
for i in range(12):
    plt.subplot(3,4, i + 1)
    img = mpimg.imread(train_df.iloc[i][43])
    plt.imshow(img)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                rotation_range=90,
                                                                brightness_range=[0, 0.1])
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# The value for class_mode in flow_from_dataframe MUST be 'raw' if you are attempting to do multilabel classification.
train_gen = train_datagen.flow_from_dataframe(train_df, 
                                              x_col='img_path', 
                                              y_col=Y_train,
                                              target_size=(150,150),
                                              class_mode='raw',
                                              batch_size=16,
                                              shuffle=True)
val_gen = val_datagen.flow_from_dataframe(val_df,
                                          x_col='img_path',
                                          y_col=Y_val,
                                          target_size=(150,150),
                                          class_mode='raw',
                                          batch_size=8)
test_gen = test_datagen.flow_from_dataframe(test_df,
                                            x_col='img_path',
                                            y_col=Y_test,
                                            target_size=(150,150),
                                            class_mode='raw')

# Modeling

To approach this problem I'm creating a U-Net model to use for our classification.

In [ ]:
def UNet(inputs):
    # First convolution block
    x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    d1_con = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(x)
    d1 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)(d1_con)
    
    # Second convolution block
    d2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d1)
    d2_con = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d2)
    d2 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)(d2_con)
    
    # Third convolution block
    d3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d2)
    d3_con = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d3)
    d3 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)(d3_con)
    
    # Fourth convolution block
    d4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d3)
    d4_con = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d4)
    d4 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)(d4_con)
    
    # Bottleneck layer
    b = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(d4)
    b = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(b)
    
    # First upsampling block
    u1 = tf.keras.layers.Conv2DTranspose(512, 3, strides =(2,2),padding='same')(b)
    u1 = tf.keras.layers.Concatenate(axis=3)([u1, d4_con])
    u1 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u1)
    u1 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u1)
    
    # Second upsampling block
    u2 = tf.keras.layers.Conv2DTranspose(256, 3, strides =(2,2),padding='valid')(u1)
    u2 = tf.keras.layers.Concatenate(axis=3)([u2, d3_con])
    u2 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u2)
    u2 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u2)
    
    # Third upsampling block
    u3 = tf.keras.layers.Conv2DTranspose(128, 3, strides =(2,2),padding='valid')(u2)
    u3 = tf.keras.layers.Concatenate(axis=3)([u3, d2_con])
    u3 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u3)
    u3 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u3)
    
    # Fourth upsampling block
    u4 = tf.keras.layers.Conv2DTranspose(64, 3, strides =(2,2),padding='same')(u3)
    u4 = tf.keras.layers.Concatenate(axis=3)([u4, d1_con])
    u4 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u4)
    u4 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(u4)
    
    # Flatten and output
    flat = tf.keras.layers.Flatten()(u4)
    out = tf.keras.layers.Dense(units=unq_disease, activation='sigmoid')(flat)
    model = tf.keras.Model(inputs=[inputs], outputs=[out])
    return model

For the actual training, we'll use area under the curve (AUC) of the receiving operator characteristic (ROC) since it was one of the suggested metrics for the multilabel classification task. ROC is the default value for the curve parameter, so we just initialize the AUC metric from keras without passing any parameters to it.

In [ ]:
auc = tf.keras.metrics.AUC(multi_label=True,thresholds=[0,0.5])
aucpr = tf.keras.metrics.AUC(curve='PR',multi_label=True,thresholds=[0,0.5])
inputs = tf.keras.layers.Input(shape=(150,150,3))
unet = UNet(inputs)
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc, aucpr])
unet.summary()

In [ ]:
unet.fit(train_gen, epochs=5, validation_data=val_gen)

In [ ]:
unet.evaluate(test_gen)

# Glossary
Not entirely necessary to know all of this for the classification tasks, but I wanted to know what all of these abbreviations stood for so I figured I'd put them down here. All definitions come from the source paper "Retinal Fundus Multi-Disease Image Dataset (RFMiD): A Dataset for Multi-Disease Detection Research."

* DR - Diabetic retinopathy
* ARMD - Age-related macular degeneration 
* MH - Media haze
* DN - Drusen
* MYA - Myopia
* BRVO - Branch retinal vein occlusion
* TSLN - Tessellation 
* ERM - Epiretinal membrane 
* LS - Laser scars 
* MS - Macular scars 
* CSR - Central serous retinopathy 
* ODC - Optic disc cupping 
* CRVO - Central retinal vein occlusion 
* TV - Tortuous vessels
* AH - Asteroid hyalosis 
* ODP - Optic disc pallor
* ODE - Optic disc edema
* ST - Optociliary shunt
* AION - Anterior ischemic optic neuropathy
* PT - Parafoveal telangiectasia
* RT - Retinal traction
* RS - Retinitis
* CRS - Chorioretinitis
* EDN - Exudation
* RPEC - Retinal pigment epithelium changes
* MHL - Macular hole
* RP - Retinitis pigmentosa
* CWS - Cotton-wool spots
* CB - Coloboma
* ODPM - Optic disc pit maculopathy
* PRH - Preretinal hemorrhage
* MNF - Myelinated nerve fibers
* HR - Hemorrhagic retinopathy
* CRAO - Central retinal artery occlusion
* TD - Tilted disc
* CME -Cystoid macular edema
* PTCR - Post-traumatic choroidal rupture
* CF - Choroidal folds
* VH - Vitreous hemorrhage
* MCA - Macroaneurysm
* VS - Vasculitis
* BRAO - Branch retinal artery occlusion
* PLQ - Plaque
* HPED - Hemorrhagic pigment epithelial detachment
* CL - Collateral
